In [ ]:
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
import plotly.graph_objects as go
import ipywidgets as widgets
import holoviews as hv
hv.extension('bokeh')
from ipywidgets import interact, interactive, fixed, interact_manual
from pathlib import Path
from dotenv import load_dotenv
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
from libs.p1_library_lk import *

In [ ]:
subset8_df['SAMPLE_YEAR_MONTH'] = pd.to_datetime(subset8_df['SAMPLE_DATE']).dt.strftime('%Y-%m')
subset8_df

In [3]:
def lineplot_8cocs(subset8_df):
    subset8_grouped = subset8_df.groupby(['SAMPLE_YEAR_MONTH','CHEMICAL_NAME']).mean()
    subset8_grouped.fillna('0')
    eight_coc_lineplot = subset8_grouped['REPORT_RESULT_VALUE'].hvplot.line(
        x='SAMPLE_YEAR_MONTH',
        groupby='CHEMICAL_NAME',
        title='Volatile Chemical Measurements in the Passaic River Basin',
        xlabel='sample year and month',
        ylabel='average concentration (ug/g)',
        rot=90)
    return eight_coc_lineplot

In [4]:
eight_coc_lineplot = lineplot_8cocs(subset8_df)

In [5]:

def call_30_df(filepath):
    subset30_df = chemical_filter(filepath, contaminant_list_30)
    # Convert all values to standardized measurements
    subset30_df = chemical_to_moles(subset30_df)
    subset30_df = subset30_df.fillna('0')
    # Add sampling year
    subset30_df['SAMPLE_YEAR'] = pd.to_datetime(subset30_df['SAMPLE_DATE']).dt.year
    subset30_df['SAMPLE_YEAR_MONTH'] = pd.to_datetime(subset30_df['SAMPLE_DATE']).dt.strftime('%Y-%m')
    return subset30_df

In [6]:
filepath = '../data/cleandata'
subset30_df = call_30_df(filepath)
more_filtered_columns = subset30_df[["SAMPLE_YEAR","CHEMICAL_NAME", "VALUE_MUMOL_PER_GRAM"]]
mean_concentrations_set30 = more_filtered_columns.groupby(["SAMPLE_YEAR","CHEMICAL_NAME"]).mean()
mean_concentrations_set30

VALUE_MUMOL_PER_GRAM
SAMPLE_YEAR CHEMICAL_NAME                                                      
1981        2,3,7,8-Tetrachlorodibenzo-p-dioxin                    1.720657e-07
            Hexachlorobiphenyl; 3,3',4,4',5,5'- (PCB 169)          1.177683e-07
            Pentachlorobiphenyl; 2',3,4,4',5- (PCB 123)            1.976103e-06
            Pentachlorobiphenyl; 2,3',4,4',5- (PCB 118)            1.009624e-04
            Pentachlorobiphenyl; 2,3,3',4',6- (PCB 110)            1.148897e-04
...                                                                         ...
2019        Pentachlorobiphenyl; 2,3,4,4',5- (PCB 114)             3.745328e-06
            Pentachlorobiphenyl; 3,3',4,4',5- (PCB 126)            5.553746e-07
            p,p'-DDD                                               4.487154e-04
            p,p'-DDE                                               6.896708e-04
            p,p'-DDT                                               3.006794e-04

[442 rows x 1 columns]

In [7]:
def average_concentration_bar():
    concentration_bar_plot = mean_concentrations_set30["VALUE_MUMOL_PER_GRAM"].hvplot.bar(
        x='SAMPLE_YEAR', 
        y='VALUE_MUMOL_PER_GRAM', 
        xlabel='Year', 
        ylabel='Average Concentration (umol/g)', 
        title= 'Average Concentration By Year', 
        groupby='CHEMICAL_NAME')
    return concentration_bar_plot
subset30_bar = average_concentration_bar()
#subset30_bar

def average_concentration_line():
    concentration_line_plot = mean_concentrations_set30["VALUE_MUMOL_PER_GRAM"].hvplot.line(
        x='SAMPLE_YEAR', 
        y='VALUE_MUMOL_PER_GRAM', 
        xlabel='Year', 
        ylabel='Average Concentration (umol/g)', 
        title= 'Average Concentration By Year', 
        groupby='CHEMICAL_NAME')
    return concentration_line_plot
subset30_line = average_concentration_line()

In [8]:
many_contaminants_column = pn.Column("# Contaminant Measurements in the Passaic River", subset30_bar, subset30_line)
metals_column = pn.Column("# Metals as Contaminants in Passaic River", copper_overlay_fig, lead_overlay_fig, mercury_overlay_fig)

In [9]:
tabs = pn.Tabs(
    ("All Industry", industry_fig),
    ("Historical Contamination", many_contaminants_column),
    ("8 COC Measurements",eight_coc_lineplot),
    ("DDT",DDT_overlay_fig), 
    ("Dioxin",dioxin_overlay_fig),
    ("PCB",pcb_overlay_fig),
    ("PAH",pah_overlay_fig),
    ("Copper",copper_overlay_fig),
    ("Lead",lead_overlay_fig),
    ("Mercury",mercury_overlay_fig)
)
tabs.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'7d0be792-7a0b-4a55-acd7-35fa53eff639': {'defs': …

Tabs
    [0] Plotly(Figure, name='All Industry')
    [1] Column
        [0] Markdown(str)
        [1] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='CHEMICAL_NAME', options=['2,3,7,8-Tetrachlorodiben...], value='2,3,7,8-Tetrachlorodibenz..., width=250)
                [1] VSpacer()
        [2] Row
            [0] HoloViews(DynamicMap)
            [1] Column
                [0] WidgetBox
                    [0] Select(margin=(20, 20, 20, 20), name='CHEMICAL_NAME', options=['2,3,7,8-Tetrachlorodiben...], value='2,3,7,8-Tetrachlorodibenz..., width=250)
                [1] VSpacer()
    [2] Row(name='8 COC Measurements')
        [0] HoloViews(DynamicMap, name='8 COC Measurements')
        [1] Column
            [0] WidgetBox
                [0] Select(margin=(20, 20, 20, 20), name='CHEMICAL_NAME', options=['2,3,7,8-Tetrachlorodiben...], value='2,3,7,8-Tetrachlorodibenz..., width=250)
            [1] VSpacer()
    [3] Plotly(Figure, name='DDT')
    [4] Plotly(Figure, name='Dioxin')
    [5] Plotly(Figure, name='PCB')
    [6] Plotly(Figure, name='PAH')
    [7] Plotly(Figure, name='Copper')
    [8] Plotly(Figure, name='Lead')
    [9] Plotly(Figure, name='Mercury')

In [10]:
DDT_sorted = DDT_df.sort_values("VALUE_MUGRAM_PER_GRAM", ascending=False)
DDT_sorted.head(20)

,SAMPLE_DATE,TASK_CODE,ANALYTIC_METHOD,CAS_RN,CHEMICAL_NAME,REPORT_RESULT_VALUE,REPORT_RESULT_UNIT,REPORT_RESULT_LIMIT,DETECT_FLAG,REPORTABLE_RESULT,LONGITUDE,LATITUDE,LOC_NAME,VALUE_MUGRAM_PER_GRAM,VALUE_MUMOL_PER_GRAM,SAMPLE_YEAR,SAMPLE_YEAR_MONTH
127486,2017-11-03 09:20:00,2017-2019 OU2 PDI Sediment,E1699,50-29-3,"p,p'-DDT",71100000.0,pg/g,2680.0,Y,Yes,-74.125310,40.741531,0,71.10,0.200564,2017,2017-11
59081,2017-11-03 09:20:00,2017 Passaic,E1699,50-29-3,"p,p'-DDT",71100000.0,pg/g,2680.0,Y,Yes,-74.125310,40.741531,0,71.10,0.200564,2017,2017-11
127486,2017-11-03 09:20:00,2017-2019 OU2 PDI Sediment,E1699,50-29-3,"p,p'-DDT",71100000.0,pg/g,2680.0,Y,Yes,-74.125310,40.741531,0,71.10,0.200564,2017,2017-11
121895,2017-11-13 15:07:00,2017 Passaic,E1699,50-29-3,"p,p'-DDT",69400000.0,pg/g,73200.0,Y,Yes,-74.144135,40.735634,0,69.40,0.195769,2017,2017-11
147427,2017-11-13 15:07:00,2017-2019 OU2 PDI Sediment,E1699,50-29-3,"p,p'-DDT",69400000.0,pg/g,73200.0,Y,Yes,-74.144135,40.735634,0,69.40,0.195769,2017,2017-11
147427,2017-11-13 15:07:00,2017-2019 OU2 PDI Sediment,E1699,50-29-3,"p,p'-DDT",69400000.0,pg/g,73200.0,Y,Yes,-74.144135,40.735634,0,69.40,0.195769,2017,2017-11
23880,2019-05-16 14:30:00,2018-2019 OU2 PDI Treatability Study,E1699,50-29-3,"p,p'-DDT",10600000.0,pg/g,4670.0,Y,Yes,-74.144568,40.735829,Arcadis Treatability Study Lab,10.60,0.029901,2019,2019-05
7414,2019-05-08 11:45:00,2019 OU2 PDI Porewater Passive Sampler,E1699,50-29-3,"p,p'-DDT",10200000.0,pg/g,1780.0,Y,Yes,-74.136740,40.741760,LPR-0340-06,10.20,0.028773,2019,2019-05
23575,2019-05-16 14:00:00,2018-2019 OU2 PDI Treatability Study,E1699,50-29-3,"p,p'-DDT",10100000.0,pg/g,4840.0,Y,Yes,-74.144568,40.735829,Arcadis Treatability Study Lab,10.10,0.028491,2019,2019-05
8936,12/2/1991,1991 Core Sediment Investigation,PCB/PEST,50-29-3,"p,p'-DDT",7690.0,ug/kg,0,Y,Yes,-74.138054,40.740527,17A,7.69,0.021693,1991,1991-12


In [11]:
metals_column = "# Metals as Contaminants in Passaic River", copper_overlay_fig, lead_overlay_fig, mercury_overlay_fig)

tabs = pn.Tabs(
    ("Historical Measurements",eight_coc_lineplot), 
    ("Copper",copper_overlay_fig), 
    ("Lead",lead_overlay_fig), 
    ("Mercury",mercury_overlay_fig), 
    ("Dioxin",dioxin_overlay_fig), 
    ("Dieldrin",dieldrin_overlay_fig), 
    ("DDT",DDT_overlay_fig), 
    ("PAH",pah_overlay_fig), 
    ("PCB",pcb_overlay_fig)
)
tabs

SyntaxError: invalid syntax (<ipython-input-11-7a6b53bd873b>, line 1)